# Executing DFT simulations using ASE, and post-processing the output via McStas

### Authors:
 - Mads Bertelsen (ESS)
 - Mousumi Upadhyay Kahaly (ELI-ALPS)
 - Shervin Nourbakhsh (ILL)
 - Gergely Nagy (ELI-ALPS)

This notebook demonstrates a very simple, yet powerful, integrated workflow of executing a DFT-based crystal structure relaxation, and then using the result in a simulated neutron-scattering experiment. The DFT calculation is conducted via the open-source *[Quantum Espresso](https://www.quantum-espresso.org)* software, and the neurton experiment is simulated via *[McStas](https://www.mcstas.org)*. 

Two Python module is used to seamlessly interface between these softwares and the notebook. The *[Atomistic Simulation Environment](https://wiki.fysik.dtu.dk/ase/index.html)* (***ASE***), a very powerful tool for atomistic simulations in general, is used to obtain the initial structure from a public database, [*COD*](https://www.crystallography.net/cod/) and communicate with Quantum Espresso; and the [***McStasScript*** module](https://github.com/PaNOSC-ViNYL/McStasScript) is used to set up and execute the McStas simulation. 


## Initial setup

In [ ]:
import sys
import os

If your Quantum-Espresso binaries are in a local folder, add it to `$PATH` here. If they are already there, you can skip the cell below.

In [ ]:
QE_bin_path = os.environ["HOME"]+"/PANOSC/bin"
os.environ['PATH']=os.environ['PATH']+":"+QE_bin_path

mcstas_outdir = "mcstas_output"
os.environ['PATH']=os.environ['PATH']+":/usr/lib64/mpich/bin:"

In [ ]:
print(os.environ['PATH'])

### Set the path for the temporary files

This folder will be used for temporary files created while running the simulations.

In [ ]:
tmpdir='/tmp/jupiter/'
print('Create temporary directory: '+tmpdir)
os.makedirs(tmpdir,exist_ok=True)
os.chdir(tmpdir)
os.makedirs(mcstas_outdir,exist_ok=True)

## Set up the input files for ASE and Quantum Espresso
ASE can take in input several file formats.
In this demo we will check that the conversion is done properly and that the output of the Quantum-Espresso (QE) simulation can be carried on with them.

In the following only CIF files will be considered as inputs for the simulation workflow


### Convert CIF to QE input file
ASE is able to convert from different formats. If you plan to run QE as a standalone package you need to use files in its input format, so you need to convert for example a CIF file into QE format. This can be done in the following way
```ase convert myfile.cif myfile.pwi```

If you run the simulation using ASE, this step is not needed since conversions are done internally and transparently 


### Download the structure 

First we download a selected cif file from the Crystallography Open Database. (Of course, this could also be done manually.)

In [ ]:
CIF_file = '1527603.cif'
print('Downloading CIF file '+CIF_file+' from crystallography.net')
os.system("wget -c https://www.crystallography.net/cod/"+CIF_file)

### Download the pseudo potential for Nitrogen

Quantum Espresso needs a suitable pseudopotential. We get it from their collection at 

In [ ]:
pseudopotfile = 'N.pbe-n-kjpaw_psl.1.0.0.UPF'
pseudo_dir = tmpdir+"/pseudo/"
os.makedirs(pseudo_dir,exist_ok=True)
os.chdir(pseudo_dir)
os.system("wget -c https://www.quantum-espresso.org/upf_files/"+pseudopotfile)
os.system("wget -c https://raw.githubusercontent.com/PaNOSC-ViNYL/workshop2020/team2/demo/team2/N.pbe-n-radius_5.UPF")
os.chdir(tmpdir)
pseudopotfile = 'N.pbe-n-radius_5.UPF'

### Check the list of files in the current working directory

In [ ]:
os.listdir()

# Setup the simulation

First, we read the CIF file and display the (initial) structure.

In [ ]:
from ase import io, Atom, Atoms
atomCIF = io.read(CIF_file)

print(atomCIF)
print(atomCIF.get_positions())

In [ ]:
from ase.visualize import view
view(atomCIF)

Then, we setup the Quantum Espresso calculation. The parameters correspond to the ones found in the input files of pw.x, for which the documentation is available [here](https://www.quantum-espresso.org/Doc/INPUT_PW.html).

In [ ]:

from ase.calculators.espresso import Espresso

In [ ]:
pseudopotentials={'N': pseudopotfile}

calc = Espresso(pseudopotentials=pseudopotentials,
                tstress=True, 
                tprnfor=True, 
                kpts=(6, 6, 6),
                ecutrho=480,
                ecutwfc=60,
                ibrav=0,
                nat=8,
                ntyp=1,
                calculation='relax',
                occupations='smearing',
                smearing='cold',
                degauss=0.001,
                outdir=tmpdir,
                pseudo_dir=pseudo_dir,
                conv_thr=1e-7,
                mixing_mode='plain',
                electron_maxstep=80,
                mixing_beta=0.5,
                ion_dynamics='bfgs',
)

atom = atomCIF
atom.calc = calc

#atom.set_calculator(calc)
#atom.get_potential_energy()
#fermi_level = calc.get_fermi_level()

### Calculate the potential energy

This takes some time, about 15 minutes.
When the calculation starts, two additional files are created:
 - espresso.pwi: QE input file with atomic structure and parameters
 - espresso.pwo: QE output 
 
ASE will parse the espresso.pwo file, and update the `atoms` object accordingly.

In [ ]:
potential_energy = atom.get_potential_energy()

print("Total energy: {0} eV".format(potential_energy))
print("Total energy: {0} eV".format(atom.get_total_energy()))
fermi_level = calc.get_fermi_level()
print("Fermi energy: {0} eV".format(fermi_level))

### Compare atom positions before and after calculation

In [ ]:
# first read the output of the QE calculation, index=-1 allow to read only the last set of positions (those at convergence)
atomsOUT = io.read('espresso.pwo',index=-1)
atomsOUT.get_positions() - atom.get_positions()

### output the result to CIF format

In [ ]:
ase_outfile = 'output.cif'
hklfile=ase_outfile+'.hkl'
io.write(ase_outfile, atom)
os.listdir()

In [ ]:
os.system('cif2hkl '+ase_outfile)

In [ ]:
os.listdir()

# Neutron scattering experiment simulation

The optimized structure is now ready. From this point, the data can be used in whatever way we desire. 

As an example, we use McStas to simulate a neutron-scattering experiment using the optimiized structure.

## Get McStasScript


In [ ]:
os.system("git clone git@github.com:PaNOSC-ViNYL/McStas_ViNYL_concept.git")

In [ ]:
os.chdir('McStas_ViNYL_concept')
#os.chdir(tmpdir)
os.makedirs(mcstas_outdir,exist_ok=True)
os.listdir()

If your McStas is installed in a different location, update the paths below accordingly. 

In [ ]:
import McStasCalculator
import McStasParameters
import math
from mcstasscript.interface import instr, plotter
from mcstasscript.interface import functions
my_configurator = functions.Configurator()
my_configurator.set_mcrun_path("/usr/local/bin/")
my_configurator.set_mcstas_path("/usr/local/mcstas/2.6/")

In [ ]:
Instr = instr.McStas_instr("powder_diffractometer")

Instr.add_parameter("wavelength", value=1.2, comment="[AA]")

src = Instr.add_component("Source", "Source_Maxwell_3")
src.xwidth = 0.12
src.yheight = 0.12
src.Lmin = "wavelength*0.94" # Simulate wavelengths in small band around requested wavelength
src.Lmax = "wavelength*1.06"
src.dist = 3.0
src.focus_xw = guide_width = 0.04
src.focus_yh = guide_height = 0.08

# Set source spectrum to ILL
src.T1 = 683.7
src.I1 = 0.5874E13
src.T2 = 257.7
src.I2 = 2.5099E13
src.T3 = 16.7
src.I3 = 1.0343E12

guide = Instr.add_component("guide", "Guide_gravity", AT=[0,0,3.0], RELATIVE="Source")
guide.w1 = guide_width
guide.h1 = guide_height
guide.l = guide_length = 10 # 10 m long guide
guide.m = 3.0
guide.G = -9.82 # Gravity

Instr.add_component("guide_end", "Arm", AT=[0, 0, guide_length], RELATIVE="guide")
Instr.add_component("mono_pos", "Arm", AT=[0, 0, 0.2], RELATIVE="guide_end")

Instr.add_parameter("mono_Q", value=3.355, comment="Monochromator scattering vector length (PG) [AA^-1]")
Instr.add_declare_var("double", "mono_theta")
Instr.add_declare_var("double", "wavevector")

# Calculate wavevector and find theta from Q = 2k sin(theta)
Instr.append_initialize("wavevector = 2*PI/wavelength;")
Instr.append_initialize("mono_theta = RAD2DEG*asin(0.5*mono_Q/wavevector);")

mono = Instr.add_component("mono", "Monochromator_curved", AT=[0,0,0], RELATIVE="mono_pos")
mono.Q = "mono_Q"
mono.height = 0.1
mono.zwidth = 0.03
mono.NH = 3 
mono.NV = 11
mono.RV = 1.5 # Focusing 
mono.set_ROTATED([0, "mono_theta", 0], RELATIVE="mono_pos")

Instr.add_component("mono_out", "Arm", AT=[0,0,0], ROTATED=[0, "mono_theta", 0], RELATIVE="mono")

L_mon = Instr.add_component("L_mon", "L_monitor", AT=[0, 0, 1.0], RELATIVE="mono_out")
L_mon.Lmin = "wavelength*0.94"
L_mon.Lmax = "wavelength*1.06"
L_mon.filename = '"L_mon.dat"'
L_mon.xwidth = 0.1
L_mon.yheight = 0.1
L_mon.nL = 150

sample = Instr.add_component("sample", "PowderN", AT=[0, 0, 1.5], RELATIVE="mono_out")
sample.radius = 0.008
sample.yheight = 0.03
sample.reflections = '"../output.cif.hkl"'
sample.barns = 0 # output.cif.hkl cross section read as fm^2 

# Wish to focus on the detector, specify height and radius for use in focusing.
detector_height = 0.2
detector_radius = 1.0
sample.d_phi = math.atan(detector_height/detector_radius)*180/3.14159
sample.tth_sign = 1.0
sample.set_SPLIT(1000)

# Set up a banana monitor to measure scattering pattern
monitor = Instr.add_component("monitor", "Monitor_nD", RELATIVE="sample")
monitor.xwidth = 2.0*detector_radius
monitor.yheight = detector_height
monitor.options = '"banana, theta limits=[10,170], bins=320"'
monitor.filename = '"banana.dat"'


In [ ]:
%%capture # the output of McStas is verbose and unneccesary for us.

data = Instr.run_full_instrument(ncount=5E6, mpi=4,
                                 foldername=mcstas_outdir, increment_folder_name=True)

# Instr.show_instrument() Uncomment to view instrum



In [ ]:
plotter.make_sub_plot(data)